In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
text = """Vernon Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere."""


In [9]:
doc = nlp(text)

In [10]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Vernon Dursley PERSON
Grunnings ORG
Dursley PERSON
Dursleys PERSON
Dudley PERSON


In [11]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import json
def load_data(file):
    with open(file, "r", encoding = "utf-8") as f:
        data = json.load(f)
    return data

In [17]:
def generate_characters(file):
    data = load_data(file)
    new_characters = []
    for item in data:
        new_characters.append(item)
    
    for item in data:
        item = item.replace("the", "").replace(
            "The", "").replace("and", "").replace("And", "")
        names = item.split(" ")
        for name in names:
            name = name.strip()
            print(name)
    #print(hp_chars)



In [18]:
characters = generate_characters("hp_characters.json")

Hannah
Abbott
Ludo
Bagman
Bathilda
Bagshot
Katie
Bell
Cuthbert
Binns
Phineas
Nigellus
Black
Sirius
Black
Walburga
Black
Amelia
Bones
Susan
Bones
Terry
Boot
Lavender
Brown
Millicent
Bulstrode
Charity
Burbage
Frank
Bryce
Alecto
Carrow
Amycus
Carrow
Reginald
Cattermole
Mary
Cattermole
Cho
Chang
Penelope
Clearwater
Mrs.
Cole
Michael
Corner
Crabbe
Vincent
Crabbe
Colin
Creevey
Dennis
Creevey
Dirk
Cresswell
Barty
Crouch
Sr
Barty
Crouch
Jr
Roger
Davies
John
Dawlish
Fleur
Delacour
Gabrielle
Delacour
Dedalus
Diggle
Amos
Diggory
Cedric
Diggory
Elphias
Doge
Antonin
Dolohov
Aberforth
Dumbledore
Albus
Dumbledore
Ariana
Dumbledore
Kendra
Dumbledore
Percival
Dumbledore
Dudley
Dursley
Marge
Dursley
Petunia
Dursley
Vernon
Dursley
Marietta
Edgecombe
Arabella
Figg
Argus
Filch
Justin
Finch-Fletchley
Seamus
Finnigan
Marcus
Flint
Mundungus
Fletcher
Filius
Flitwick
Florean
Fortescue
Cornelius
Fudge
Marvolo
Gaunt
Merope
Gaunt
Morfin
Gaunt
Anthony
Goldstein
Goyle
Gregory
Goyle
Hermione
Granger
Gregorovitch
Fenr

In [19]:
import spacy
import json
import random


In [20]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)


def save_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],
                    [annotations],
                    drop=0.2,
                    sgd=optimizer,
                    losses=losses
                )
            print(losses)
    return (nlp)


In [21]:
TRAIN_DATA = load_data("hp_training_data.json")
nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("hp_ner_model")


ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.ner.EntityRecognizer object at 0x00000211BC69BE20> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.